In [34]:
import numpy as np
import itertools
def logical_error_rate(p_phy, d):
    if d%2 == 1:
        return 0.1 * (p_phy / 0.01)**(0.5 * d + 0.5)
    else:
        return 0.05 * (d*d)/(d-1)/(d-1) * (p_phy / 0.01)**(0.5 * d)
    
def logical_error_retangle(dz, dx, p_phy):
    logical_error_z = 0.5 * (dx/dz) * logical_error_rate(p_phy, dz)
    logical_error_x = 0.5 * (dz/dx) * logical_error_rate(p_phy, dx)
    return logical_error_z, logical_error_x

def distillation_code_distance(eps, p_phy):
    for d in range(3,50,2):
        if 6.5 * d * logical_error_rate(p_phy, d) < eps:
            return d

# def logical_error_rate(code_distance, physical_error_rate=0.001):
#     return 0.1*(physical_error_rate/0.01)**((code_distance+1)/2)

# def code_distance(logical_error_rate, physical_error_rate=0.001):
#     exponent = np.log(logical_error_rate*10)/np.log(physical_error_rate/0.01)
#     return max(int(np.ceil(exponent))*2-1, 1)

def search_t_protocol(target_error_rate, p_phy):
    t_error_range = []
    initial_t_error_range = []
    if p_phy == 0.001:
        eps = [0.0004, 8e-05, 7e-06, 5e-06, 3e-06, 2e-07, 3e-08, 1e-08, 6e-09, 2e-09, 1e-09, 
             3.8850000000000016e-10, 3.304716553287983e-10, 1.2955e-12, 1.2178655692729767e-12, 4.405000000000001e-14,
             3.563400713436386e-14, 4.815000000000002e-15, 3.9081789802289295e-15, 1.2225000000000002e-15, 1.1252984389348027e-15, 
             1.3475000000000002e-16, 5.0750000000000023e-17, 4.03781224489796e-17]
        volume = [1000, 4000, 5000, 6000, 8000, 20000, 30000, 40000, 50000, 60000, 100000, 
             2199074, 2523632, 4685822, 5216768, 5827026, 6438000, 7131094, 7827712, 8609258, 9397136, 10272750, 10432750, 11317504]
    elif p_phy == 0.0005:
        eps=[1e-5, 8e-7,3e-07, 2e-07, 1e-08, 1e-09, 4e-11, 3.7260546875000005e-12, 3.2233414127423824e-12, 1.186650390625e-12, 
             1.1583162379535146e-12,2.9983642578125002e-15, 2.91156640625e-15, 3.871166992187501e-17, 3.3973240312071337e-17, 
             7.554645996093752e-20,6.170946930280958e-20]
        volume=[2000, 3500, 4000, 4500, 8000.0, 10000.0, 20000.0, 1621006, 1888000, 2185074, 2509632, 3688250.0, 4144784.0, 
                4645822.0, 5176768.0,7051094.0, 7747712.0]
    
    for i, t_error in enumerate(eps): 
        s = 8 * t_error ** 2
        if s < 1.5 * target_error_rate:
            t_error_range.append([eps[i],volume[i]])
        if len(t_error_range) >= 6:
            break
    for i, t_error in enumerate(eps): 
        s = t_error
        if s < 1.5 * target_error_rate:
            initial_t_error_range.append([eps[i],volume[i]])
        if len(initial_t_error_range) >= 6:
            break
    return t_error_range, initial_t_error_range #length = 6


def MEK(d, t_error, rl_1_error, ml_error, t_volume, rl_1_volume, ml_volume, p_phy): # Campbell-1603.04230
    # ml_error 输入态的error
    # rl_1_error R l-1 门的error（包含修正）
    pass_rate = 1 - 8 * t_error - 2 * ml_error - 0.5 * rl_1_error
    pass_rate += 96 * logical_error_rate(p_phy, d)
    
    error_rate = 8 * t_error ** 2 + ml_error ** 2 + rl_1_error / 4
    error_rate += 0.5 * logical_error_rate(p_phy, d)
    
    
    depth = 20 * d
    footprint = 2 * 16 * d ** 2
    
    volume = (footprint * depth + 8 * t_volume + 2 * ml_volume + rl_1_volume) / pass_rate / 2

    return error_rate, volume

def mek_level_one(d, t_error, initial_t_error, t_volume,initial_t_volume, ll, p_phy): # d, t_error, ll>=4
    ml_error = 7 / 15 * p_phy
    ml_volume = 0
    rl_1_error = initial_t_error
    rl_1_volume = initial_t_volume
    error_rates = [initial_t_error]
    volumes = [initial_t_volume]
    for l in range(4, ll+1):
        error_rate, volume = MEK(d, t_error, rl_1_error, ml_error, t_volume, rl_1_volume, ml_volume, p_phy)
        error_rates.append(error_rate)
        volumes.append(volume)
        rl_1_error = rl_1_error * 0.5 + error_rate
        rl_1_volume = rl_1_volume * 0.5 + volume
    return error_rates, volumes

def mek_level_two(d1,d2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,t_error2, initial_t_error2, 
                  t_volume2, initial_t_volume2, ll, p_phy): 
    error_rates_one, volumes_one = mek_level_one(d1, t_error1, initial_t_error1, initial_t_volume1, t_volume1, 25, p_phy)#限定最大L=25
    error_rates = [initial_t_error2]
    volumes = [initial_t_volume2]
    rl_1_error = initial_t_error2
    rl_1_volume = initial_t_volume2
    for l in range(4, ll+1):
        ml_error = error_rates_one[l-3]
        ml_volume = volumes_one[l-3]
        error_rate, volume = MEK(d2, t_error2, rl_1_error, ml_error, t_volume2, rl_1_volume, ml_volume, p_phy)
        error_rates.append(error_rate)
        volumes.append(volume)
        rl_1_error = rl_1_error * 0.5 + error_rate
        rl_1_volume = rl_1_volume * 0.5 + volume
    return error_rates, volumes

def mek_level_three(d1,d2,d3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,t_error2, initial_t_error2, 
                    t_volume2, initial_t_volume2,  t_error3, initial_t_error3, t_volume3, initial_t_volume3, ll, p_phy): 
    error_rates_two, volumes_two = mek_level_two(d1,d2, t_error1, initial_t_error1, initial_t_volume1, t_volume1,t_error2,
                                                 initial_t_error2, initial_t_volume2, t_volume2, 25, p_phy)#限定最大L=25
    error_rates = [initial_t_error3]
    volumes = [initial_t_volume3]
    rl_1_error = initial_t_error3
    rl_1_volume = initial_t_volume3
    for l in range(4, ll+1):
        ml_error = error_rates_two[l-3]
        ml_volume = volumes_two[l-3]
        error_rate, volume = MEK(d2, t_error2, rl_1_error, ml_error, t_volume2, rl_1_volume, ml_volume, p_phy)
        error_rates.append(error_rate)
        volumes.append(volume)
        rl_1_error = rl_1_error * 0.5 + error_rate
        rl_1_volume = rl_1_volume * 0.5 + volume
    return error_rates, volumes

def search_mek_level_two(ll, target_error_rate, p_phy):#搜索t_error，d用梯度下降
    parameters = ["fail",float("inf")] 
    t_error_range2, initial_t_error_range2 = search_t_protocol(target_error_rate, p_phy)
    t_error_range1, initial_t_error_range1 = search_t_protocol(target_error_rate**0.5, p_phy)
    for t_error1, t_volume1 in t_error_range1:
        for initial_t_error1, initial_t_volume1 in initial_t_error_range1:
            for t_error2, t_volume2 in t_error_range2:
                for initial_t_error2, initial_t_volume2 in initial_t_error_range2:
                    d_target = [41,41] #d1,d2
                    d_step = [1,1] #搜索步长
                    for i in range(len(d_target)):
                        d_range0 = [41,41]
                        for j in range(d_target[i]//d_step[i]):
                            d1,d2= d_range0
                            error_rates, volumes = mek_level_two(d1,d2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,
                                                                 t_error2, initial_t_error2, t_volume2, initial_t_volume2, ll, p_phy)
                            if error_rates[-1] < target_error_rate:
                                d_target[i] = d_range0[i]
                                d_range0[i] -= d_step[i]
                            else:
                                break
                    d1,d2 = d_target
                    d_set = list(itertools.product([d1,d1+2],[d2,d2+2]))
                    for dd1, dd2 in d_set:
                        error_rates, volumes = mek_level_two(dd1,dd2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,
                                                         t_error2, initial_t_error2, t_volume2, initial_t_volume2, ll, p_phy)
                        if error_rates[-1] < target_error_rate and volumes[-1] < parameters[-1]:
                                parameters = [dd1,dd2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,
                                              t_error2, initial_t_error2, t_volume2, initial_t_volume2, error_rates[-1], volumes[-1]]
    return parameters
    
    
    
def search_mek_level_three(ll, target_error_rate, p_phy):#搜索t_error，d用梯度下降
    parameters = ["fail",float("inf")] 
    t_error_range3, initial_t_error_range3 = search_t_protocol(target_error_rate, p_phy)
    t_error_range2, initial_t_error_range2 = search_t_protocol(target_error_rate**0.5, p_phy)
    t_error_range1, initial_t_error_range1 = search_t_protocol((target_error_rate**0.5)**0.5, p_phy)
    for t_error1, t_volume1 in t_error_range1:
        for initial_t_error1, initial_t_volume1 in initial_t_error_range1:
            for t_error2, t_volume2 in t_error_range2:
                for initial_t_error2, initial_t_volume2 in initial_t_error_range2:
                    for t_error3, t_volume3 in t_error_range3:
                        for initial_t_error3, initial_t_volume3 in initial_t_error_range3:
                            d_target = [41,41,41] #d1,d2
                            d_step = [1,1,1] #搜索步长
                            for i in range(len(d_target)):
                                d_range0 = [41,41,41]
                                for j in range(d_target[i]//d_step[i]):
                                    d1,d2,d3= d_range0
                                    error_rates, volumes = mek_level_three(d1,d2,d3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,
                                                                           t_error2, initial_t_error2, t_volume2, initial_t_volume2,  t_error3, 
                                                                           initial_t_error3, t_volume3, initial_t_volume3, ll, p_phy)
                                    if error_rates[-1] < target_error_rate:
                                        d_target[i] = d_range0[i]
                                        d_range0[i] -= d_step[i]
                                    else:
                                        break
                            d1,d2,d3 = d_target
                            d_set = list(itertools.product([d1,d1+2],[d2,d2+2],[d3,d3+2]))
                            for dd1, dd2, dd3 in d_set:
                                error_rates, volumes = mek_level_three(dd1,dd2,dd3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,
                                                                           t_error2, initial_t_error2, t_volume2, initial_t_volume2,  t_error3, 
                                                                           initial_t_error3, t_volume3, initial_t_volume3, ll, p_phy)
                                if error_rates[-1] < target_error_rate and volumes[-1] < parameters[-1]:
                                        parameters = [dd1,dd2,dd3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,
                                                                           t_error2, initial_t_error2, t_volume2, initial_t_volume2,  t_error3, 
                                                                           initial_t_error3, t_volume3, initial_t_volume3, error_rates[-1], volumes[-1]]
    return parameters

In [25]:
np.pi/2**12,np.pi/2**7, np.pi/2**9, np.pi/2**25

(0.0007669903939428206,
 0.02454369260617026,
 0.006135923151542565,
 9.362675707309822e-08)

In [3]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-12, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 9.323070074911609e-13 5935354.856282866
5 9.92773809701025e-13 8319532.407568745
6 8.968789864090455e-13 11049669.005112411
7 8.61451692858505e-13 13574631.156403974
8 9.065540551914748e-13 16226986.134095533
9 9.463197979874926e-13 19006383.228704613
10 9.806417659969662e-13 21912884.844523806
11 9.978352154165215e-13 25126542.21948627
12 9.90716428482739e-13 28878042.641512465
13 8.218726150788152e-13 34081834.639264464
14 8.420814884802837e-13 37743974.27925465
15 8.588375182183858e-13 41534091.98287816
16 8.726061627339391e-13 45452288.09274603
17 8.838350390777448e-13 49498663.03155925
18 8.929341527211688e-13 53673317.30211494
19 9.002669524310033e-13 57976351.48725948
20 9.061481099271291e-13 62407866.24981598
21 9.108452640971737e-13 66967962.332503565
22 9.145829105021532e-13 71656740.55785938
23 9.17547268149807e-13 76474301.82816969
24 9.198913994777064e-13 81420747.12541339
25 9.217401570823107e-13 86496177.51121926


In [2]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-12, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)


4 9.103414907426698e-13 2809525.7219301634
5 7.978610791141012e-13 3747955.3615733115
6 8.72819805072941e-13 4705375.713829362
7 9.401723887777299e-13 5693789.864059412
8 9.991181290473867e-13 6713205.392127753
9 9.504721220991922e-13 7780644.409942243
10 9.911341894213873e-13 8888114.277798833
11 9.99476133560076e-13 10765267.065367242
12 8.769753587928336e-13 13010426.934259374
13 8.775312200478859e-13 14549345.487878555
14 7.161694015457578e-13 14686363.833560463
15 7.199895301729789e-13 16112718.758389398
16 7.230709144743589e-13 17583715.776902262
17 7.255442257973096e-13 19099366.257820893
18 7.275209463005363e-13 20659681.572888587
19 7.290948075421507e-13 22264673.09685872
20 7.303437043060949e-13 23914352.20748187
21 7.313317582324339e-13 25608730.28549269
22 7.321113379146911e-13 27347818.71459747
23 7.327249289171489e-13 29131628.88146278
24 7.332068028691749e-13 30960172.175705366
25 7.335844694318611e-13 32833459.989883486


In [4]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-9, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 9.992660490449385e-10 1285910.6893036363
5 9.772063250050622e-10 1980826.2543885268
6 9.579182826522613e-10 2796185.935368079
7 9.982764333535528e-10 3629765.877249147
8 8.811955312564242e-10 4928564.803230946
9 8.401265240781025e-10 5861464.967493751
10 8.594849879258826e-10 6826484.612840274
11 8.718356109954797e-10 7823910.669527167
12 8.841551925263459e-10 8853626.285251355
13 8.939929495374237e-10 9915815.223166836
14 9.018209192406063e-10 11010478.80332807
15 9.080297535503431e-10 12137618.349662669
16 9.129401197233607e-10 13297235.191102693
17 9.168133952460399e-10 14489330.661977015
18 9.198613375615751e-10 15713906.101947028
19 9.222545954305298e-10 16970962.855677065
20 9.241300494873755e-10 18260502.272366956
21 9.255970425996769e-10 19582525.70522925
22 9.267426003739583e-10 20937034.51096277
23 9.276357596044135e-10 22324030.049253076
24 9.283311256865835e-10 23743513.682316355
25 9.2887177470275e-10 25195486.77449408


In [5]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-9, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 8.739023297264202e-10 498900.15626140457
5 9.9796922284615e-10 907058.9396785971
6 9.642009177563714e-10 1296825.738581559
7 9.232012047711445e-10 1657737.6658366732
8 9.969501590429027e-10 2025624.7135735664
9 8.920775142515376e-10 2994884.6052382933
10 9.045287026260405e-10 3544217.3316796795
11 9.138135739394673e-10 4123071.473634731
12 9.207384001036253e-10 4731454.351166841
13 9.259036376043151e-10 5369373.288944169
14 9.297566944701545e-10 6036835.614606195
15 9.326310672991063e-10 6733848.657705364
16 9.347754134691851e-10 7460419.7490547495
17 9.36375171928217e-10 8216556.2203572085
18 3.653706139927807e-10 8958674.336425738
19 3.6699877046649127e-10 9620655.652996035
20 3.683266386114263e-10 10294571.115581932
21 3.6940262638129546e-10 10980417.081850285
22 3.7026969546209555e-10 11678189.913070444
23 3.7096506342010075e-10 12387885.973729543
24 3.715203931939213e-10 13109501.631191164
25 3.719622457378285e-10 13843033.25539881


In [6]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-15, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 fail inf
5 fail inf
6 fail inf
7 fail inf
8 fail inf
9 fail inf
10 fail inf
11 fail inf
12 fail inf
13 fail inf
14 fail inf
15 fail inf
16 fail inf
17 fail inf
18 fail inf
19 fail inf
20 fail inf
21 fail inf
22 fail inf
23 fail inf
24 fail inf
25 fail inf


In [7]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-15, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 fail inf
5 fail inf
6 fail inf
7 fail inf
8 fail inf
9 fail inf
10 fail inf
11 fail inf
12 fail inf
13 fail inf
14 fail inf
15 fail inf
16 fail inf
17 fail inf
18 fail inf
19 fail inf
20 fail inf
21 fail inf
22 fail inf
23 fail inf
24 fail inf
25 fail inf


In [8]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_three(ll = l, target_error_rate = 1e-15, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 8.439846065895109e-16 56079250.889510356
5 9.167009472781182e-16 104653646.21945538
6 9.8755574384119e-16 164613875.28093535
7 9.44991577426079e-16 236812709.33878466
8 9.99802687171487e-16 320669969.7709414
9 8.011490703740045e-16 421734168.0261643
10 8.615175472054175e-16 529444524.4642771
11 9.228234935250678e-16 648826569.9038798
12 9.827444497588717e-16 779900858.1630558
13 9.933617316428546e-16 923677867.2073131
14 8.067843604211631e-16 1101494111.773857
15 8.201500514311596e-16 1271433856.3818
16 8.319961451104251e-16 1453420056.9137862
17 8.42358804410346e-16 1647477761.8544402
18 8.513215095297482e-16 1853631946.1811016
19 8.589970720935495e-16 2071907512.3853543
20 8.655135392645863e-16 2302329291.360427
21 8.710037448768889e-16 2544922043.1701756
22 8.755979921374187e-16 2799710457.7158346
23 8.794192672199023e-16 3066719155.3159585
24 8.825804021995515e-16 3345972687.213625
25 8.851826758055333e-16 3637495536.023291


In [9]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_three(ll = l, target_error_rate = 1e-15, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 5.06384800872203e-16 29193417.509963434
5 fail inf
6 3.679086187698823e-16 86251953.01811187
7 4.0201495404321215e-16 123781628.92048627
8 4.275996442214797e-16 167349470.05521846
9 4.467929705156482e-16 216964320.9774774
10 4.611924207519985e-16 272635021.9538027
11 4.719959902410594e-16 334370409.02519965
12 4.801021304790416e-16 402179314.11684453
13 4.861846853372178e-16 476070565.16729146
14 4.907490729806803e-16 556052986.2606946
15 4.941744076792488e-16 642135397.7525706
16 4.967450813632003e-16 734326616.3841646
17 4.986744436938205e-16 832635455.3833469
18 5.0012255866067e-16 937070724.5517141
19 5.012095203470718e-16 1047641230.3385625
20 5.02025439197367e-16 1164355775.9029362
21 5.026379318303348e-16 1287223161.165158
22 5.030977388760879e-16 1416252182.8493018
23 5.034429389978708e-16 1551451634.5179853
24 5.037021100995157e-16 1692830306.6007373
25 5.03896700899936e-16 1840396986.41705


In [10]:
for l in range(4, 25):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-13, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 7.756905163092865e-14 3992782.6647231868
5 9.62863244274412e-14 5191887.876813458
6 9.398478767020683e-14 6505455.3937657755
7 4.198536480956279e-14 8598982.338957548
8 4.449907149867788e-14 10149725.215688327
9 4.6353394598165306e-14 11762941.272689719
10 4.772093480316494e-14 13438647.517332021
11 4.872919712442532e-14 15176860.961630471
12 4.947235416694131e-14 16977598.62223125
13 5.00199449458463e-14 18840877.520403508
14 5.042330683638601e-14 20766714.682035487
15 5.0720330735772606e-14 22755127.1376333
16 5.09389760145739e-14 24806131.922321532
17 5.1099868266767745e-14 26919746.075844843
18 5.121821883176852e-14 29095986.642570242
19 5.130524287518075e-14 31334870.67148964
20 5.136920679685684e-14 33636415.21622254
21 5.141620168272261e-14 36000637.33501871
22 5.145071432119965e-14 38427554.0907608
23 5.147604866440263e-14 40917182.55096679
24 5.149463682500642e-14 43469539.78779233


In [11]:
sum([np.float64(4132198.9673255403), np.float64(5484545.057549819), np.float64(7169455.520702402), 
 np.float64(9011242.639582045), np.float64(10966866.502004256),np.float64(15185539.716602026), np.float64(17445458.477995954)]),sum([4149889.7572538718,
5380698.513351309,
6668075.030034606,
8012032.532553306,
9412584.248533763,
12383523.244963653,
13980465.762841025])

(np.float64(69395306.88176204), 59987269.08953153)

In [28]:
#T decomposition
def T_decomposition(l, p_phy, infidelity):
    if p_phy == 0.001:
        eps = [0.0004, 8e-05, 7e-06, 5e-06, 3e-06, 2e-07, 3e-08, 1e-08, 6e-09, 2e-09, 1e-09, 
                 3.8850000000000016e-10, 3.304716553287983e-10, 1.2955e-12, 1.2178655692729767e-12, 4.405000000000001e-14,
                 3.563400713436386e-14, 4.815000000000002e-15, 3.9081789802289295e-15, 1.2225000000000002e-15, 1.1252984389348027e-15, 
                 1.3475000000000002e-16, 5.0750000000000023e-17, 4.03781224489796e-17]
        volume = [1000, 4000, 5000, 6000, 8000, 20000, 30000, 40000, 50000, 60000, 100000, 
                 2199074, 2523632, 4685822, 5216768, 5827026, 6438000, 7131094, 7827712, 8609258, 9397136, 10272750, 10432750, 11317504]
    elif p_phy == 0.0005:
        eps=[1e-5, 8e-7,3e-07, 2e-07, 1e-08, 1e-09, 4e-11, 3.7260546875000005e-12, 3.2233414127423824e-12, 1.186650390625e-12, 
                 1.1583162379535146e-12,2.9983642578125002e-15, 2.91156640625e-15, 3.871166992187501e-17, 3.3973240312071337e-17, 
                 7.554645996093752e-20,6.170946930280958e-20]
        volume=[2000, 3500, 4000, 4500, 8000.0, 10000.0, 20000.0, 1621006, 1888000, 2185074, 2509632, 3688250.0, 4144784.0, 
                    4645822.0, 5176768.0,7051094.0, 7747712.0]

    #infidelity = 1e-13
    t_count = 3 * (l - 1 - np.log(np.pi)/np.log(2))
    output1, output2 = 0, 0
    for i,e in enumerate(eps):
        if e * t_count <= infidelity * (l-2):
            #print(e)
            output1, output2 = e * t_count, t_count * volume[i]
            break
    #if output1==0:
        #output1, output2 = e * t_count, t_count * volume[-1]
    return output1, output2
        


In [29]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(l=l, p_phy=0.0005, infidelity=1e-12)
    print(l,"("+str(volumes)+")")

4 (14920858.201421158)
5 (25985608.20142116)
6 (37050358.20142116)
7 (48115108.20142116)
8 (59179858.201421164)
9 (70244608.20142117)
10 (81309358.20142117)
11 (92374108.20142117)
12 (103438858.20142117)
13 (114503608.20142117)
14 (125568358.20142117)
15 (136633108.20142117)
16 (147697858.20142117)
17 (158762608.20142117)
18 (169827358.20142117)
19 (180892108.20142114)
20 (191956858.20142114)
21 (203021608.20142114)
22 (214086358.20142114)
23 (225151108.20142114)
24 (236215858.20142114)
25 (247280608.20142114)


In [30]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(l=l, p_phy=0.001, infidelity=1e-12)
    print(l,"("+str(volumes)+")")

4 (23573301.343996294)
5 (41054379.343996294)
6 (58535457.343996294)
7 (76016535.34399629)
8 (93497613.3439963)
9 (110978691.3439963)
10 (128459769.3439963)
11 (145940847.34399632)
12 (163421925.34399632)
13 (180903003.34399632)
14 (198384081.34399632)
15 (215865159.34399632)
16 (233346237.34399632)
17 (250827315.34399632)
18 (268308393.34399632)
19 (285789471.3439963)
20 (303270549.3439963)
21 (320751627.3439963)
22 (338232705.3439963)
23 (355713783.3439963)
24 (373194861.3439963)
25 (390675939.3439963)


In [31]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(l=l, p_phy=0.0005, infidelity=1e-9)
    print(l,error_rates, volumes)

4 1.6182046446332172e-10 80910.23223166086
5 2.818204644633217e-10 140910.23223166086
6 4.018204644633217e-10 200910.23223166086
7 5.218204644633217e-10 260910.23223166086
8 6.418204644633218e-10 320910.2322316609
9 7.618204644633218e-10 380910.2322316609
10 8.818204644633218e-10 440910.2322316609
11 1.0018204644633218e-09 500910.2322316609
12 1.1218204644633218e-09 560910.2322316609
13 1.2418204644633218e-09 620910.2322316609
14 1.3618204644633218e-09 680910.2322316609
15 1.4818204644633218e-09 740910.2322316609
16 1.6018204644633218e-09 800910.2322316609
17 1.7218204644633218e-09 860910.2322316609
18 1.8418204644633218e-09 920910.2322316609
19 1.9618204644633216e-09 980910.2322316607
20 2.0818204644633216e-09 1040910.2322316607
21 2.2018204644633216e-09 1100910.2322316607
22 2.3218204644633216e-09 1160910.2322316607
23 2.4418204644633216e-09 1220910.2322316607
24 2.5618204644633216e-09 1280910.2322316607
25 2.6818204644633216e-09 1340910.2322316607


In [32]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(l=l, p_phy=0.001, infidelity=1e-9)
    print(l,error_rates, volumes)

4 1.5716812611000128e-09 8896379.40173037
5 2.7371812611000134e-09 15493601.40173037
6 3.902681261100014e-09 22090823.40173037
7 4.311171816890908e-09 32922070.55936254
8 5.302586782877304e-09 40492966.559362546
9 6.294001748863698e-09 48063862.559362546
10 7.285416714850093e-09 55634758.559362546
11 8.276831680836488e-09 63205654.559362546
12 9.268246646822883e-09 70776550.55936255
13 1.0259661612809277e-08 78347446.55936255
14 1.1251076578795674e-08 85918342.55936255
15 1.2242491544782069e-08 93489238.55936255
16 1.3233906510768463e-08 101060134.55936255
17 1.4225321476754858e-08 108631030.55936255
18 1.5216736442741255e-08 116201926.55936255
19 1.6208151408727646e-08 123772822.55936253
20 1.719956637471404e-08 131343718.55936253
21 1.8190981340700436e-08 138914614.55936253
22 1.918239630668683e-08 146485510.55936253
23 2.0173811272673225e-08 154056406.55936253
24 2.116522623865962e-08 161627302.55936253
25 2.2156641204646015e-08 169198198.55936253


In [33]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(l=l, p_phy=0.0005, infidelity=1e-15)
    print(l,error_rates, volumes)   

4 1.566085101727154e-16 18794726.846347958
5 2.7274351993834044e-16 32732192.846347958
6 3.888785297039655e-16 46669658.84634796
7 5.050135394695905e-16 60607124.84634796
8 6.211485492352156e-16 74544590.84634796
9 7.372835590008407e-16 88482056.84634796
10 8.534185687664657e-16 102419522.84634796
11 9.695535785320907e-16 116356988.84634796
12 1.0856885882977158e-15 130294454.84634796
13 1.2018235980633408e-15 144231920.84634796
14 1.317958607828966e-15 158169386.84634796
15 1.4340936175945909e-15 172106852.84634796
16 1.5502286273602158e-15 186044318.84634796
17 1.666363637125841e-15 199981784.84634796
18 1.782498646891466e-15 213919250.84634796
19 1.8986336566570907e-15 227856716.84634793
20 2.014768666422716e-15 241794182.84634793
21 2.130903676188341e-15 255731648.84634793
22 2.2470386859539657e-15 269669114.8463479
23 2.363173695719591e-15 283606580.8463479
24 2.479308705485216e-15 297544046.8463479
25 2.5954437152508408e-15 311481512.8463479


In [17]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(l=l, p_phy=0.001, infidelity=1e-15)
    print(l,error_rates, volumes)

4 5.451326896608151e-16 41558529.40788971
5 9.493826896608152e-16 72376779.40788971
6 1.3536326896608152e-15 103195029.40788971
7 1.7578826896608152e-15 134013279.40788971
8 2.1621326896608156e-15 164831529.40788972
9 2.566382689660816e-15 195649779.40788972
10 2.9706326896608157e-15 226468029.40788972
11 3.374882689660816e-15 257286279.40788972
12 3.779132689660816e-15 288104529.4078897
13 4.183382689660816e-15 318922779.4078897
14 4.587632689660816e-15 349741029.4078897
15 4.991882689660816e-15 380559279.4078897
16 5.396132689660816e-15 411377529.4078897
17 5.800382689660816e-15 442195779.4078897
18 6.204632689660816e-15 473014029.4078897
19 6.6088826896608156e-15 503832279.40788966
20 7.013132689660816e-15 534650529.40788966
21 7.417382689660816e-15 565468779.4078896
22 7.821632689660816e-15 596287029.4078896
23 8.225882689660815e-15 627105279.4078896
24 8.630132689660815e-15 657923529.4078896


In [39]:
3 * (30- 1 - np.log(np.pi)/np.log(2))

np.float64(82.04551161158304)